# Customer Churn Prediction
### We can understand this is a classification problem and we must group the customers into different catagories of churn risk(1 to 5)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Fetching Data

In [4]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,fffe430049004400390031003300,Wan Garraway,60,M,GBH67OC,Village,Platinum Membership,2015-10-06,Yes,CID27405,...,571.450000,7778.92,5.0,704.93,Yes,Yes,No,Not Applicable,Poor Website,-1
1,fffe43004900440031003500360039003900,Carmelo Keitt,48,F,PN9XX93,Village,Silver Membership,2015-12-25,No,xxxxxxxx,...,259.620000,12549.93,36.204800566565154,765.97,Yes,No,No,Not Applicable,Poor Product Quality,3
2,fffe43004900440033003200360037003200,Albertha Fragale,28,M,NI00GZV,Town,No Membership,2015-04-19,?,CID25795,...,160.240000,25494.34,14.0,663.76,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,fffe43004900440035003900320030003000,Sidney Hakes,44,F,EFOP88Q,Town,Basic Membership,2017-04-13,Yes,CID61848,...,1025.913200,4578.77,Error,663.69,No,Yes,No,Not Applicable,Poor Customer Service,4
4,fffe43004900440031003600340032003900,Zoe Rickman,12,M,7E58K5C,City,Basic Membership,2017-07-04,Yes,CID63467,...,1079.755553,1436.20,Error,617.71,Yes,No,No,Not Applicable,Poor Customer Service,5


In [5]:
test.head()

,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440033003200320035003600,Tobias Liebold,24,F,I4AYTC2,City,Premium Membership,2015-04-22,No,xxxxxxxx,...,17,101.50,32593.20,15.0,801.18,Yes,No,No,Not Applicable,Products always in Stock
1,fffe43004900440032003200350035003400,Patrick Kizer,53,F,WV0LB6W,Town,Silver Membership,2016-01-19,No,xxxxxxxx,...,12,324.61,39155.49,21.0,NaN,No,Yes,No,Not Applicable,No reason specified
2,fffe43004900440031003000380038003300,Annamaria Freese,53,F,94O1F22,Town,No Membership,2016-02-07,Yes,CID19334,...,20,47.71,35434.17,12.0,675.17,Yes,No,No,Not Applicable,Poor Product Quality
3,fffe43004900440034003200300031003800,Gilda Lundy,61,M,74WFG9K,NaN,Gold Membership,2017-10-24,No,xxxxxxxx,...,19,451.66,30621.93,7.0,755.93,Yes,Yes,Yes,Solved,Poor Product Quality
4,fffe43004900440034003100380030003300,Angla Alameda,46,F,249HVEX,Town,Premium Membership,2016-06-11,No,xxxxxxxx,...,12,266.68,50462.15,Error,806.67,Yes,Yes,Yes,Solved,Products always in Stock


## Preprocessing

In [6]:
train.columns

Index(['customer_id', 'Name', 'age', 'gender', 'security_no',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'referral_id', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback', 'churn_risk_score'],
      dtype='object')

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29593 entries, 0 to 29592
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   29593 non-null  object 
 1   Name                          29593 non-null  object 
 2   age                           29593 non-null  int64  
 3   gender                        29593 non-null  object 
 4   security_no                   29593 non-null  object 
 5   region_category               25227 non-null  object 
 6   membership_category           29593 non-null  object 
 7   joining_date                  29593 non-null  object 
 8   joined_through_referral       29593 non-null  object 
 9   referral_id                   29593 non-null  object 
 10  preferred_offer_types         29365 non-null  object 
 11  medium_of_operation           29593 non-null  object 
 12  internet_option               29593 non-null  object 
 13  l

In [8]:
train.describe()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,churn_risk_score
count,29593.000000,29593.000000,29593.000000,29593.000000,26879.000000,29593.000000
mean,37.069611,-42.039604,243.931773,29345.208786,687.410044,3.460312
std,15.899136,229.080617,399.512766,19483.727165,195.291908,1.409662
min,10.000000,-999.000000,-2814.109110,800.460000,-760.661236,-1.000000
25%,23.000000,8.000000,60.370000,14159.340000,616.485000,3.000000
50%,37.000000,12.000000,161.600000,27644.160000,698.000000,4.000000
75%,51.000000,16.000000,357.790000,41013.490000,764.475000,5.000000
max,64.000000,26.000000,3040.410000,99914.050000,2069.069761,5.000000


## Assumptions
### By business knowledge we can say the columns Customer_id,Name,security_no,referral_id do not have any impact on churn

In [ ]:
train.drop(["customer_id","Name","security_no","referral_id"],axis="columns",inplace=True)

In [12]:
train.head()

,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,60,M,Village,Platinum Membership,2015-10-06,Yes,Gift Vouchers/Coupons,?,Fiber_Optic,19:39:26,...,571.450000,7778.92,5.0,704.93,Yes,Yes,No,Not Applicable,Poor Website,-1
1,48,F,Village,Silver Membership,2015-12-25,No,Without Offers,Smartphone,Mobile_Data,16:01:26,...,259.620000,12549.93,36.204800566565154,765.97,Yes,No,No,Not Applicable,Poor Product Quality,3
2,28,M,Town,No Membership,2015-04-19,?,Credit/Debit Card Offers,Smartphone,Wi-Fi,12:49:57,...,160.240000,25494.34,14.0,663.76,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,44,F,Town,Basic Membership,2017-04-13,Yes,Gift Vouchers/Coupons,?,Mobile_Data,23:45:49,...,1025.913200,4578.77,Error,663.69,No,Yes,No,Not Applicable,Poor Customer Service,4
4,12,M,City,Basic Membership,2017-07-04,Yes,Without Offers,Smartphone,Mobile_Data,13:05:12,...,1079.755553,1436.20,Error,617.71,Yes,No,No,Not Applicable,Poor Customer Service,5


In [15]:
train["joined_through_referral"].value_counts()

joined_through_referral
No     12686
Yes    12568
?       4339
Name: count, dtype: int64

In [ ]:
train[train["joined_through_referral"] =="Yes" or train["joined_through_referral"]=="No"]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().